In [217]:
import openseespy.opensees as ops
import opsvis as opsv
import os
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display

In [218]:
if not os.path.exists('Data'): 
    os.mkdir('Data')

ops.wipe()  # Clear opensees model

In [219]:
# Defining base units for the simulation.
m = 1.0  # Meter (base unit for length)
s = 1.0  # Second (base unit for time)
kg = 1.0  # Kilogram (base unit for mass)

# Derived units for force and pressure.
N = kg * m / s ** 2  # Newton (unit of force)
Pa = N / m ** 2  # Pascal (unit of pressure)

# Additional conversions for different units commonly used in structural engineering.
inches = 0.0254 * m  # Conversion factor for inches to meters.
ft = 12 * inches  # Conversion factor for feet to meters.
kip = 4448.2216152548 * N  # Conversion factor for kips (1000 pounds-force) to Newtons.
ksi = 6.895 * 10 ** 6 * Pa  # Conversion factor for ksi (1000 psi) to Pascals.

In [220]:
# # Create an IntSlider widget
# slider = widgets.IntSlider(value=0, min=0, max=100, step=1, description='Column Height')

# # Function to update the variable based on slider value
# def update_variable(change):
#     global colL
#     colL = change['new']
#     # print(f'Updated colL: {colL}')

# # Observe changes in the slider's value
# slider.observe(update_variable, names='value')

# # Display the slider
# display(slider)

# # description='Column Length:'

### Inputs

In [221]:
colL = widgets.FloatText(
    value=36*ft,
    description='Column Height (m):',
    disabled=False,
    style={'description_width': 'initial'}
)

# Cross-sectional dimensions
b = widgets.FloatText(
    value=5*ft,
    description='Column breadth (m):',
    disabled=False,
    style={'description_width': 'initial'}
) # Breadth

d = widgets.FloatText(
    value=5*ft,
    description='Column depth (m):',
    disabled=False,
    style={'description_width': 'initial'}
) # Depth

E = widgets.FloatText(
    value=4227 * ksi,
    description='Elastic modulus (Pa):',
    disabled=False,
    style={'description_width': 'initial'}
)

Py = widgets.FloatText(
    value=-2000.0 * kip,
    description='Vertical load (N):',
    disabled=False,
    style={'description_width': 'initial'}
)

disp = widgets.FloatText(
    value=100 * inches,
    description='Horizontal displacement (m):',
    disabled=False,
    style={'description_width': 'initial'}
)

out_inputs = widgets.Output(layout={'border': '1px solid black'})

button = widgets.Button(description="Run Analysis", disabled = False)


@out_inputs.capture(clear_output = True, wait = True)
def model_definition_general(change):

    button.disabled = False

    # Nodal coordinates
    n1 = (0.0, 0.0) # Use floating point values
    n2 = (0.0, colL.value)

    # Cross sectional area
    A = b.value * d.value # Area

    # Moment of Inertia
    Iz = (b.value * d.value ** 3) / 12

    Px = 1 # Lateral load

    g = 9.81 # Gravitational acceleration (N / kg)

    # Compute nodal mass
    mass_x = abs(Py.value) / g # kg
    # print(f'The nodal mass value corresponding to the horizontal DOF is {mass_x:.0f} kg') # 
    massValues = [mass_x, 1 * 10 ** -9, 0.0] # ndf nodal mass values corresponding to each DOF


    # Model Definition

    ops.wipe()  # Clear opensees model

    ops.model('basic', '-ndm', 2, '-ndf', 3)    # 2D model with 3 degrees of freedom per node.

    # Create nodes
    nodal_crds = (n1, n2)
    for nodeTag, crds in enumerate(nodal_crds, start = 1): # https://docs.python.org/3/library/functions.html#enumerate
        ops.node(nodeTag, *crds)

    # Boundary Conditions - single point constraints
    constrValues = [1, 1, 1]
    ops.fix(1, *constrValues)   # Node 1 is a fixed connection
    # ops.fix(2, 0, 0, 0)         # Node 2 is free

    # Define geometric transformation: performs a linear geometric transformation of beam stiffness and resisting force from the local-coordinate system to the global-coordinate system
    ops.geomTransf('Linear', 1)     #  associate a tag of 1 to transformation

    # connectivity: (make A very large, 10e6 times its actual value)
    # element elasticBeamColumn eleTag iNode jNode A E Iz transfTag
    ops.element('elasticBeamColumn', 1, 1, 2, A * 10 ** 6, E.value, Iz, 1)     # element elasticBeamColumn 1 1 2 3600000000 4227 1080000 1;

    # nodal masses: Time - 23:30, 32:20
    ops.mass(2, *massValues)     #  node ,  Mass=Weight/g.

    opsv.plot_model()
    plt.title('Model')
    plt.ylabel('Height')
    plt.show()


colL.observe(model_definition_general, names='value')
b.observe(model_definition_general, names='value')
d.observe(model_definition_general, names='value')
E.observe(model_definition_general, names='value')
Py.observe(model_definition_general, names='value')
disp.observe(model_definition_general, names='value')

display(colL, b, d, E, Py, disp, out_inputs)

FloatText(value=10.9728, description='Column Height (m):', style=DescriptionStyle(description_width='initial')…

FloatText(value=1.5239999999999998, description='Column breadth (m):', style=DescriptionStyle(description_widt…

FloatText(value=1.5239999999999998, description='Column depth (m):', style=DescriptionStyle(description_width=…

FloatText(value=29145165000.0, description='Elastic modulus (Pa):', style=DescriptionStyle(description_width='…

FloatText(value=-8896443.2305096, description='Vertical load (N):', style=DescriptionStyle(description_width='…

FloatText(value=2.54, description='Horizontal displacement (m):', style=DescriptionStyle(description_width='in…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [222]:
output_analysis = widgets.Output(layout={'border': '1px solid black'})

display(button, output_analysis)

@output_analysis.capture(clear_output = True)
def on_button_clicked(b):
    run_analysis(Py.value, disp.value)
    button.disabled = True

button.on_click(on_button_clicked)


def run_analysis(Py, disp):

    # Define RECORDERS -------------------------------------------------------------

    # https://openseespydoc.readthedocs.io/en/latest/src/nodeRecorder.html#node-recorder-command
    # recorder('Node', '-file', filename, '-time', '-node', *nodeTags=[], '-dof', *dofs=[], respType)
    dofs = [1, 2, 3]
    ops.recorder('Node', '-file', 'Data/DispFreeEx1aPush.out', '-time', '-node', 2, '-dof', *dofs, 'disp')     #  displacements of free node
    ops.recorder('Node', '-file', 'Data/DispBaseEx1aPush.out', '-time', '-node', 1, '-dof', *dofs, 'disp')     #  displacements of support node
    ops.recorder('Node', '-file', 'Data/ReacBaseEx1aPush.out', '-time', '-node', 1, '-dof', *dofs, 'reaction')     #  support reaction

    # https://openseespydoc.readthedocs.io/en/latest/src/elementRecorder.html#element-recorder-command
    # recorder('Element', '-file', filename, '-time', '-ele', *eleTags=[], '-eleRange', startEle, endEle, '-region', regionTag, *args)
    ops.recorder('Element', '-file', 'Data/FColEx1aPush.out', '-time', '-ele', 1, 'globalForce')     #  element forces -- column
    ops.recorder('Element', '-file', 'Data/DColEx1aPush.out', '-time', '-ele', 1, 'deformation')     #  element deformations -- column


    # Define GRAVITY Loads-------------------------------------------------------------
    ops.timeSeries('Linear', 1)  # timeSeries Linear 1;

    # Define Load Pattern
    ops.pattern('Plain', 1, 1)
    ops.load(2, 0.0, Py, 0.0)   #  node , FX FY MZ -- superstructure-weight


    ops.wipeAnalysis()     # adding this to clear Analysis module 
    ops.system('BandGeneral')     #  how to store and solve the system of equations in the analysis
    ops.numberer('Plain')     #  renumber dofs to minimize band-width (optimization), if you want to
    ops.constraints('Plain')     #  how it handles boundary conditions
    ops.integrator('LoadControl',0.1)     #  determine the next time step for an analysis,   apply gravity in 10 steps
    ops.algorithm('Newton')     #  use Newtons solution algorithm: updates tangent stiffness at every iteration
    ops.analysis('Static')     #  define type of analysis static or transient

    ops.test('NormDispIncr',1.0e-8,6)     #  determine if convergence has been achieved at the end of an iteration step
    ops.analyze(10)     #  perform gravity analysis
    ops.loadConst('-time',0.0)     #  hold gravity constant and restart time


    # sfac = opsv.plot_defo(sfac = 100)
    # plt.title('Deformed shape of column after gravity loads have been applied')
    # plt.show()
    
    # define LATERAL load -------------------------------------------------------------
    # Lateral load pattern
    ops.timeSeries('Linear', 2)     # timeSeries Linear 2;
    # define Load Pattern
    Px = 1
    ops.pattern('Plain', 2, 2) # 
    ops.load(2, Px, 0.0, 0.0)    #  node , FX FY MZ -- REPRESENTATIVE lateral load at top node

    # pushover: diplacement controlled static analysis
    # Note we do not wipe the analysis

    # disp = 100 * inches
    # print(f'Horizontal displacement at top of column is set to be: {disp} metres')
    steps = 1000 # Must be an integer
    incr = disp / steps
    ops.integrator('DisplacementControl', 2, 1, incr)     #  switch to displacement control, for node 2, dof 1, 0.1 increment
    ops.analyze(steps)


    sfac = opsv.plot_defo(sfac = 1)
    plt.title(f'Deformed shape of column (Scale factor: {sfac})')

    opsv.plot_loads_2d(nep=17, sfac=False, fig_wi_he=False, fig_lbrt=False, 
                   fmt_model_loads={'color': 'black', 'linestyle': 'solid', 'linewidth': 1.2, 'marker': '', 'markersize': 1}, 
                   node_supports=True, truss_node_offset=0, ax=False)
    plt.title('Loads applied to Column (N)')
    plt.show()
    
    info = ops.printModel()

Button(description='Run Analysis', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…